# Problems 3 - text_to_sql_llamaindex_groq

In [31]:
# %pip install sqlalchemy
# %pip install llama-index
# %pip install llama-index-llms-groq
# %pip install llama-index-embeddings-huggingface
# %pip install llama-parse


In [32]:
from sqlalchemy import MetaData, create_engine
from llama_index.core import SQLDatabase
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [33]:
engine = create_engine("sqlite:///scorecards.db")
metadata = MetaData()
sql_database = SQLDatabase(engine)
print("Tables in the db: \n", sql_database.get_usable_table_names())


Tables in the db: 
 ['batting', 'bowling']


In [34]:
with open('key.txt', 'r') as f:
    GROQ_API_KEY = f.read().strip()


In [35]:
llm = Groq(
    model="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY,
)


In [36]:
#Setup Embedding Model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [37]:
#Define Global Settings Configuration
Settings.llm = llm
Settings.embed_model = embed_model
#In LlamaIndex, you can define global settings so you don't have to pass
#the LLM / embedding model objects everywhere.

In [38]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=sql_database.get_usable_table_names(),
    llm=llm,
    api_key=GROQ_API_KEY,
    response_mode="no_synthesis",  # Skip heavy response synthesis
    similarity_top_k=5,
    max_rows=15
)


In [39]:
questions = [
    "Who were the top run-scorers for both teams?",
    "Which bowler took the most wickets in the match?",
    "How did Virat Kohli perform in this match?",
    "Which pair of batsman made the highest runs in a partnership?",
    "Which team made the highest score as part of opening partnership?",
    "How many catches did Kohli take?",
    "What was the final score and who won the match?",
    "How many overs did each team bat?",
    "What was the difference in run rates between the two teams?",
    "How many extras were conceded by each team?"
]

query_df = pd.DataFrame({
    "Question": questions
})

print(query_df)


                                            Question
0       Who were the top run-scorers for both teams?
1   Which bowler took the most wickets in the match?
2         How did Virat Kohli perform in this match?
3  Which pair of batsman made the highest runs in...
4  Which team made the highest score as part of o...
5                   How many catches did Kohli take?
6    What was the final score and who won the match?
7                  How many overs did each team bat?
8  What was the difference in run rates between t...
9        How many extras were conceded by each team?


In [40]:
response_df = query_df.copy()
response_df["sql_query"] = None
response_df["response"] = None

for i in range(len(query_df)):
    response = query_engine.query(query_df.iloc[i].values[0])
    response_df["sql_query"].iloc[i] = response.metadata["sql_query"]
    response_df["response"].iloc[i] = response
    print(f"Question {i+1} responded")

response_df.to_csv("response_df.csv", index=False)


Question 1 responded
Question 2 responded
Question 3 responded
Question 4 responded
Question 5 responded
Question 6 responded
Question 7 responded
Question 8 responded
Question 9 responded
Question 10 responded


Q1

In [41]:
q_num = 0
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: Who were the top run-scorers for both teams?

SQL Query: SELECT Batting_player_name, team, runs_scored FROM batting ORDER BY runs_scored DESC

Response: Based on the SQL response, the top run-scorers for both teams are:

**India:**
1. Rohit Sharma (76 runs)
2. Daryl Mitchell is not in India, so we look for the next highest run scorer in India, which is Shreyas Iyer (48 runs) 
3. KL Rahul (34 runs)
4. Shubman Gill (31 runs)
5. Axar Patel (29 runs)
6. Hardik Pandya (18 runs)
7. Virat Kohli (1 run)

**New Zealand:**
1. Daryl Mitchell (63 runs)
2. Michael Bracewell (53 runs)
3. Rachin Ravindra (37 runs)
4. Glenn Phillips (34 runs)
5. Will Young (15 runs)
6. Tom Latham (14 runs)
7. Kane Williamson (11 runs)
8. Mitchell Santner (8 runs)
9. Nathan Smith (0 runs)


Q2

In [42]:
q_num = 1
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: Which bowler took the most wickets in the match?

SQL Query: SELECT Bowling_player_name FROM bowling ORDER BY Wicket_taken DESC LIMIT 1;

Response: Varun Chakravarthy took the most wickets in the match.


Q3

In [43]:
q_num = 2
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: How did Virat Kohli perform in this match?

SQL Query: SELECT Batting_player_name, runs_scored, strike_rate FROM batting WHERE Batting_player_name = 'Virat Kohli' ORDER BY runs_scored DESC LIMIT 1;

Response: Virat Kohli scored 1 run in this match with a strike rate of 50.


Q4

In [44]:
q_num = 3
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: Which pair of batsman made the highest runs in a partnership?

SQL Query: SELECT batting.Batting_player_name, batting2.Batting_player_name, batting.runs_scored 
FROM batting 
JOIN batting AS batting2 ON batting.innings = batting2.innings 
JOIN batting AS batting3 ON batting2.innings = batting3.innings 
WHERE batting3.runs_scored = ( SELECT MAX(batting.runs_scored) 
                              FROM batting 
                              JOIN batting AS batting2 ON batting.innings = batting2.innings 
                              JOIN batting AS batting3 ON batting2.innings = batting3.innings 
                              WHERE batting.Batting_player_name != batting2.Batting_player_name 
                              AND batting.Batting_player_name != batting3.Batting_player_name )
ORDER BY batting.runs_scored DESC;

Response: Based on the SQL response, the pair of batsmen who made the highest runs in a partnership is:

Rohit Sharma and Axar Patel, with a total of 76 runs.

H

Q5

In [45]:
q_num = 4
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: Which team made the highest score as part of opening partnership?

SQL Query: SELECT team, SUM(runs_scored) AS total_runs FROM batting WHERE innings = 1 GROUP BY team ORDER BY total_runs DESC LIMIT 1;

Response: The New Zealand team made the highest score as part of an opening partnership with 251 runs.


Q6

In [46]:
q_num = 5
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: How many catches did Kohli take?

SQL Query: To answer this question, we need to join the 'batting' and 'bowling' tables based on the player name, and then count the number of catches taken by Kohli. However, there is no direct column for catches in the given tables. We can infer that a catch is made when a batsman is dismissed and the bowler takes the wicket. 

We will assume that a batsman is dismissed when their status details contain the word 'dismissed'. We will also assume that a bowler takes a wicket when the status details of the batsman contain the word 'dismissed' and the bowler's name is mentioned in the status details.

Here is the SQL query to run:


SELECT COUNT(*) 
FROM batting AS b 
JOIN bowling AS w ON b.Batting_player_name = w.Bowling_player_name 
WHERE b.Batting_player_name = 'Kohli' AND b.Status_details LIKE '%dismissed%' AND w.Status_details LIKE '%dismissed%' AND w.Bowling_player_name = 'Kohli';


However, this query will not work because the 'bowling' ta

Q7

In [47]:
q_num = 6
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: What was the final score and who won the match?

SQL Query: SELECT SUM(runs_scored) AS total_runs, team 
FROM batting 
GROUP BY team 
ORDER BY total_runs DESC

Response: Based on the SQL response, we can determine that the final score and the winner of the match are as follows:

India scored 254 runs and New Zealand scored 251 runs. Since India scored more runs, they won the match.

Response: "India won the match with a final score of 254 runs, defeating New Zealand who scored 251 runs."


Q8

In [48]:
q_num = 7
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: How many overs did each team bat?

SQL Query: SELECT team, SUM(Overs_bowled) AS total_overs FROM bowling GROUP BY team

Response: India batted for 50 overs and New Zealand batted for 49 overs.


Q9

In [49]:
q_num = 8
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: What was the difference in run rates between the two teams?

SQL Query: SELECT team, SUM(runs_scored) / SUM(balls_faced) AS run_rate FROM batting GROUP BY team

Response: The difference in run rates between the two teams is 0.0273. 

To calculate this, we subtract the run rate of New Zealand from the run rate of India:

0.8639455782312925 - 0.8366666666666667 = 0.0273


Q10

In [50]:
q_num = 9
print('Qustion:', response_df.iloc[q_num][0])
print('\nSQL Query:', response_df.iloc[q_num][1])
print('\nResponse:', response_df.iloc[q_num][2])

Qustion: How many extras were conceded by each team?

SQL Query: SELECT team, SUM(Wide_balls + No_balls) AS extras FROM bowling GROUP BY team

Response: India conceded 9 extras, while New Zealand conceded 6 extras.
